# Testing Gradio

In [1]:
# Installing Gradio
%pip install gradio
%pip install fsspec --upgrade


[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext gradio

c:\Users\Herberto\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
%reload_ext gradio
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv("C:/Users/Herberto/Downloads/DEU_permutation_df.csv")
df_hamburg = pd.read_csv("C:/Users/Herberto/Downloads/df_hamburg.csv")
df_ahrweiler = pd.read_csv("C:/Users/Herberto/Downloads/df_ahrweiler.csv")
landkreise = df['district'].unique().tolist()

# Dummy Model
class LSTMModel:
    def __init__(self):
        pass

    def predict(self, data):
        prediction = [[data[in_landkreis_tb], data[school_close_intensity], data[workplace_close_intensity], data[cancel_events_intensity], data[gatherings_restrictions_intensity], data[transport_closing_intensity], \
                               data[stay_home_restrictions_intensity], data[internal_movements_restrictions_intensity], data[international_movements_restrictions_intensity], data[information_campgaings_intensity], \
                                data[testing_policy_intensity], data[contact_tracing_intensity], data[facial_coverings_intensity], data[vaccination_policy_intensity], data[elderly_people_protection_intensity]], \
                      compute_graph(df_hamburg), compute_graph(df_ahrweiler), compute_diff_graphs(df_hamburg, df_ahrweiler)]
        return prediction

    def current_measures(self):
        aktuelle_Maßnahmen = ["face_mask", "school_closing", "clubs_closing"]
        return aktuelle_Maßnahmen

def make_prediction(data):
    try:
        date = datetime.strptime(data[in_inzidenz_bisher_tb_start], "%Y-%m-%d")
        if date < datetime(2019, 1, 1) or date > datetime(2022, 12, 31):
            raise ValueError("Date out of range")
    except ValueError:
        return "Invalid date format or range"
        
    model = LSTMModel()
    prediction = model.predict(data)
    return prediction

def get_current_measures():
    model = LSTMModel()
    curr_measures = model.current_measures()
    return curr_measures

# Validation

def validate_inputs(landkreis, einwohnerzahl, bisherige_inz):
    try:
        date = datetime.strptime(bisherige_inz, "%Y-%m-%d")
        if date < datetime(2019, 1, 1) or date > datetime(2022, 12, 31):
            raise ValueError("Date out of range")
    except ValueError:
        return False

    return True

def validate_checkbox_choices_len(data):
    if len(data) >= 5:
        return {error_box: gr.update(value="Error: max. 4 choices", visible=True),
                submit_btn: gr.update(visible=False)}
    else:
        return {error_box: gr.update(visible=False),
                submit_btn: gr.update(visible=True)}

# Plots

def compute_graph(df_to_graph: pd.DataFrame):
    fig, ax = plt.subplots()
    x1 = df_to_graph['year'].astype(str) + '-W' + df_to_graph['week'].astype(str)
    y1 = df_to_graph['incidence'].reset_index(drop=True)

    ax.plot(x1, y1, label='Inzidenz Entwicklung in {}'.format(df_to_graph['administrative_area_level_3'].iloc[0]))
    ax.set_xlabel('Zeit')
    ax.set_ylabel('Inzidenz')
    ax.set_title('Inzidenzverlauf')
    ax.set_xticks(np.arange(len(x1)))
    ax.set_xticklabels(x1, rotation=45)
    ax.legend()
    return fig

def compute_diff_graphs(actual_course: pd.DataFrame, alternate_course: pd.DataFrame):
    fig, ax = plt.subplots()

    x1 = actual_course['year'].astype(str) + '-W' + actual_course['week'].astype(str)
    x2 = alternate_course['year'].astype(str) + '-W' + alternate_course['week'].astype(str)
    y1 = actual_course['incidence'].reset_index(drop=True)
    y2 = alternate_course['incidence'].reset_index(drop=True)

    ax.plot(x1, y1, label='Reale Entwicklung')
    ax.plot(x2, y2, label='Alternative Entwicklung')
    ax.set_xlabel('Zeit')
    ax.set_ylabel('Inzidenz')
    ax.set_title('Inzidenzverlauf')
    ax.set_xticks(range(len(x1)))
    ax.set_xticklabels(x1, rotation=45)

    ax.fill_between(x1, y1, y2, color='green', alpha=0.5)
    ax.legend()
    return fig

def compute_integral(original_graph: pd.DataFrame, alternative_graph: pd.DataFrame):
    """
    Compute integral
    :param original_graph: df of the original course
    :param alternative_graph:df of the alternative course
    :return: diff integral between both scenario
    High positive values means: alternative course is better
    High negative values means: original course is better
    """
    incidence_one = original_graph['incidence']
    incidence_two = alternative_graph['incidence']
    return np.trapz(incidence_one - incidence_two)

# Hilfsfunktionen
def mode_switch(mode):
    if "Szenarienvergleich" in mode:
        return {maßnahmen_abfrage1_col: gr.update(visible=True),
                maßnahmen_abfrage2_col: gr.update(visible=False)}
    elif "Empfehlung" in mode:
        return {maßnahmen_abfrage1_col: gr.update(visible=False),
                maßnahmen_abfrage2_col: gr.update(visible=True)}
    elif "Szenarienvergleich" in mode and "Empfehlung" in mode:
        return {maßnahmen_abfrage1_col: gr.update(visible=False),
                maßnahmen_abfrage2_col: gr.update(visible=False)}
    else:
        return {maßnahmen_abfrage1_col: gr.update(visible=False),
                maßnahmen_abfrage2_col: gr.update(visible=False)}

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # Vorhersage für Ihren Landkreis
        Geben Sie den Namen des Landkreises ein, um die Vorhersage zu erhalten.
        """
    )
    with gr.Column():
        # Inputs
        in_landkreis_tb = gr.Dropdown(landkreise, label="Landkreis")
        # in_einwohnerzahl_tb = gr.Textbox(label="Einwohnerzahl", placeholder="Geben Sie hier eine Einwohnerzahl als Ganzzahl ein...")
        gr.Markdown(
        """
        # Auswahl zwischen zwei Modi:
        ##  1. Szenarienvergleich
        ### Vergleich der Szenarien zwischen Prediction und tatsächliche Inzidenz. Die Auswahl enthält folgende Informationen:
        1. Maßnahme
        2. Intensität der Maßnahme
        3. Beschreibung der Intensität

        ## 2. Empfehlung
        ###  Angabe von Empfehlungen für einen bestimmten Zeitraum unter Berücksichtigung von max. 4 Maßnahmen
        """
        )

    # Mode Section
    #with gr.Column():
    #    scene1_cb = gr.Checkbox(label="Szenarienvergleich",info="Vergleich der Szenarien zwischen Prediction und tatsächliche Inzidenz..")
    #    scene2_cb = gr.Checkbox(label="Empfehlung",info="Angabe von Empfehlungen für einen bestimmten Zeitraum unter Berücksichtigung von max. 4 Maßnahmen")
    
    with gr.Column() as mode_decide:
        checkbox_mode_choices = gr.CheckboxGroup(["Szenarienvergleich", "Empfehlung"],
                                                   label="Modi",
                                                   info="Wählen Sie einen von beiden Modi aus...")
    # Mode 1
    with gr.Column(visible=False) as maßnahmen_abfrage1_col:
        # school_closing
        with gr.Tab("Schließung der Schulen"):
            with gr.Row():
                school_close_intensity = gr.Slider(0,3, label="Intensität der Maßnahme",step=1) 
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Schließung der Schulen

                0 - keine Maßnahmen

                1 - Empfehlung der Schließung oder Öffnung aller Schulen mit Änderungen, die zu signifikanten Unterschieden im Vergleich zum Nicht-Covid-19-Betrieb führen
            
                2 - Schließung vorschreiben (nur einige Stufen oder Kategorien, z. B. nur Gymnasien oder nur öffentliche Schulen)
            
                3 - Schließung aller Stufen erforderlich
                """
                )
        # workplace_closing
        with gr.Tab("Schließung der Arbeitsplätze"):
            with gr.Row():
                workplace_close_intensity = gr.Slider(0,3, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Schließung der Arbeitsplätze

                0 - keine Maßnahmen

                1 - Empfehlung der Schließung (oder Empfehlung der Heimarbeit) oder Öffnung aller Betriebe mit Änderungen, die zu erheblichen Unterschieden im Vergleich zum Nicht-Covid-19-Betrieb führen
            
                2 - Schließung (oder Heimarbeit) für einige Sektoren oder Kategorien von Arbeitnehmern vorschreiben
            
                3 - Schließung (oder Arbeit von zu Hause aus) für alle Arbeitsplätze, die nicht unbedingt notwendig sind (z. B. Lebensmittelgeschäfte, Ärzte)
                """
                )

         # cancel_events
        with gr.Tab("Events verbieten"):
            with gr.Row():
                cancel_events_intensity = gr.Slider(0,2, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Events verbieten

                0 - keine Maßnahmen

                1 - Annullierung empfehlen

                2 - Annullierung erforderlich
                """
            )
        
        # gatherings_restrictions
        with gr.Tab("Versammlungsrestriktionen"):
            with gr.Row():
                gatherings_restrictions_intensity = gr.Slider(0,4, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Versammlungsrestriktionen

                0 - keine Einschränkungen

                1 - Beschränkungen für sehr große Versammlungen (die Grenze liegt bei über 1000 Personen)
                
                2 - Beschränkungen für Versammlungen zwischen 101-1000 Personen
                
                3 - Beschränkungen für Versammlungen zwischen 11-100 Personen
                
                4 - Beschränkungen für Versammlungen von 10 Personen oder weniger
                """
            )

        # transport_closing
        with gr.Tab("Schließung des Verkehrs"):
            with gr.Row():
                transport_closing_intensity = gr.Slider(0,2, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Schließung des Verkehrs

                0 - keine Maßnahmen

                1 - Empfehlung der Schließung (oder erhebliche Reduzierung des Volumens/der Strecke/der verfügbaren Verkehrsmittel)

                2 - Schließung vorschreiben (oder den meisten Bürgern verbieten, sie zu benutzen)
                """
            )

        # stay_home_restrictions
        with gr.Tab("Beschränkungen des Zuhausebleibens"):
            with gr.Row():
                stay_home_restrictions_intensity = gr.Slider(0,3, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Beschränkungen des Zuhausebleibens

                0 - keine Maßnahmen

                1 - Empfehlung, das Haus nicht zu verlassen

                2 - Verbot, das Haus zu verlassen, mit Ausnahmen für tägliche Bewegung, Lebensmitteleinkäufe und "notwendige" Fahrten
                
                3 - Verbot des Verlassens des Hauses mit minimalen Ausnahmen (z. B. einmal pro Woche, oder nur eine Person darf gleichzeitig das Haus verlassen, usw.)
                """
            )

        # internal_movements_restrictions
        with gr.Tab("Bewegungseinschränkungen (intern)"):
            with gr.Row():
                internal_movements_restrictions_intensity = gr.Slider(0,2, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Bewegungseinschränkungen (intern)

                0 - keine Maßnahmen

                1 - Empfehlung, nicht zwischen Regionen/Städten zu reisen

                2 - interne Bewegungseinschränkungen sind vorhanden
                """
        )
        
        # international_movements_restrictions
        with gr.Tab("Bewegungseinschränkungen (international)"):
            with gr.Row():
                international_movements_restrictions_intensity = gr.Slider(0,4, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Bewegungseinschränkungen (international)

                0 - keine Einschränkungen

                1 - Überprüfung von Ankünften

                2 - Quarantäne für Ankünfte aus einigen oder allen Regionen

                3 - Verbot von Ankünften aus einigen Regionen

                4 - Verbot für alle Regionen oder vollständige Schließung der Grenzen
                """
        )
        
         # information_campgaings
        with gr.Tab("Informationskampagnen"):
            with gr.Row():
                information_campgaings_intensity = gr.Slider(0,2, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Informationskampagnen

                0 - keine öffentliche Informationskampagne zu Covid-19

                1 - öffentliche Beamte mahnen zur Vorsicht in Bezug auf Covid-19

                2 - koordinierte öffentliche Informationskampagne (z. B. über traditionelle und soziale Medien)
                """
        )
                
         # testing_policy
        with gr.Tab("Test Richtlinien"):
            with gr.Row():
                testing_policy_intensity = gr.Slider(0,3, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Test Richtlinien

                0 - keine Testpolitik
                
                1 - nur Personen, die a) Symptome aufweisen UND b) bestimmte Kriterien erfüllen (z. B. Mitarbeiter in Schlüsselpositionen, die ins Krankenhaus eingeliefert wurden, die mit einem bekannten Fall in Kontakt gekommen sind, die aus dem Ausland zurückgekehrt sind)
                
                2 - Tests für alle, die Covid-19-Symptome aufweisen
                
                3 - offene öffentliche Tests (z. B. "Drive-Through"-Tests für asymptomatische Personen)
                """
        )
                
          # contact_tracing
        with gr.Tab("Kontaktverfolgung"):
            with gr.Row():
                contact_tracing_intensity = gr.Slider(0,2, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
                # Beschreibung der Maßnahme: Kontaktverfolgung

              0 - keine Ermittlung von Kontaktpersonen

              1 - begrenzte Ermittlung von Kontaktpersonen; nicht bei allen Fällen durchgeführt

              2 - umfassende Ermittlung von Kontaktpersonen; durchgeführt für alle identifizierten Fälle
                """
        )
                
         # facial_coverings
        with gr.Tab("Gesichtsbedeckung"):
            with gr.Row():
                facial_coverings_intensity = gr.Slider(0,4, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
              # Beschreibung der Maßnahme: Gesichtsbedeckung

              0 - keine Richtlinie

              1 - Empfohlen

              2 - Erforderlich in einigen bestimmten gemeinsamen/öffentlichen Räumen außerhalb der Wohnung, in denen andere Personen anwesend sind, oder in einigen Situationen, in denen eine soziale Distanzierung nicht möglich ist
             
              3 - Erforderlich in allen gemeinsamen/öffentlichen Räumen außerhalb des Hauses, in denen andere Personen anwesend sind, oder in allen Situationen, in denen eine soziale Distanzierung nicht möglich ist
              
              4 - Außerhalb der Wohnung jederzeit erforderlich, unabhängig vom Ort oder der Anwesenheit anderer Personen
                """
        )
        
         # vaccination_policy
        with gr.Tab("Impfrichtlinien"):
            with gr.Row():
                vaccination_policy_intensity = gr.Slider(0,5, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
              # Beschreibung der Maßnahme: Impfrichtlinien

                0 - Keine Verfügbarkeit
                
                1 - Verfügbarkeit für EINE der folgenden Gruppen: Schlüsselkräfte/ klinisch gefährdete Gruppen (nicht ältere Menschen) / ältere Menschen
                
                2 - Verfügbarkeit für ZWEI der folgenden Gruppen: Schlüsselpersonal/ klinisch gefährdete Gruppen (nicht ältere Menschen) / ältere Menschen
                
                3 - Verfügbarkeit für ALLE der folgenden Bereiche: Schlüsselkräfte/ klinisch gefährdete Gruppen (nicht ältere Menschen) / ältere Menschen
                
                4 - Verfügbarkeit für alle drei plus teilweise zusätzliche Verfügbarkeit (Auswahl breiter Gruppen/Alter)
                
                5 - Universelle Verfügbarkeit
                """
        )
                
         # elderly_people_protection
        with gr.Tab("Schutz älterer Menschen"):
            with gr.Row():
                elderly_people_protection_intensity = gr.Slider(0,3, label="Intensität der Maßnahme",step=1)
                gr.Markdown(
                """
              # Beschreibung der Maßnahme: Schutz älterer Menschen

                0 - keine Maßnahmen
                
                1 - Empfohlene Isolierungs-, Hygiene- und Besuchsbeschränkungsmaßnahmen in LTCFs und/oder ältere Menschen sollen zu Hause bleiben
                
                2 - Geringe Einschränkungen für Isolierung, Hygiene in LTCFs, einige Einschränkungen für externe Besucher und/oder Einschränkungen zum Schutz älterer Menschen zu Hause
                
                3 - Weitreichende Beschränkungen für Isolierung und Hygiene in LTCFs, Verbot aller nicht unbedingt erforderlichen externen Besucher und/oder alle älteren Menschen müssen zu Hause bleiben und dürfen das Haus mit minimalen Ausnahmen nicht verlassen und keine externen Besucher empfangen
                """
        )
    
    #Errorbox for Mode 2
    error_box = gr.Textbox(label="Error", visible=False)

    # Mode 2
    with gr.Column(visible=False) as maßnahmen_abfrage2_col:
        checkbox_group_choices = gr.CheckboxGroup(["Schließung der Schulen",
                          "Schließung der Arbeitsplätze",
                          "Events verbieten","Versammlungsrestriktionen",
                          "Schließung des Verkehrs",
                          "Beschränkungen des Zuhausebleibens",
                          "Bewegungseinschränkungen (intern)",
                          "Bewegungseinschränkungen (international)",
                          "Informationskampagnen","Test Richtlinien",
                          "Kontaktverfolgung","Gesichtsbedeckung",
                          "Impfrichtlinien", "Schutz älterer Menschen"], label="Maßnahmen", info="Wählen Sie maximal 4 Maßnahmen aus...")
    

    # Inteferenz
    in_inzidenz_bisher_tb_start = gr.Textbox(label="Bisherige Inzidenz von ...", placeholder="Format: 2021-01-01")
    in_inzidenz_bisher_tb_start.value = "2021-01-01"
    in_inzidenz_bisher_tb_ende = gr.Textbox(label="bis ...", placeholder="Format: 2021-01-01") 
    in_inzidenz_bisher_tb_ende.value = "2021-02-01"


    # Outputs
    with gr.Column():
        out_akt_maßnahmen = gr.Textbox(label="Aktuelle Maßnahmen")
        out_prognose_tb = gr.Textbox(label="Vorschläge")
        with gr.Row():
            out_inzidenz_ahrweiler_plot = gr.Plot(label="Inzidenzen Ahrweiler")
            out_inzidenz_hamburg_plot = gr.Plot(label="Inzidenzen Hamburg")
        out_inzidenz_integral_plot = gr.Plot(label="Inzidenzen Integral")

    submit_btn = gr.Button("Start")

    # Events
    ## Visibility Mode Changes
    checkbox_mode_choices.change(mode_switch,inputs=checkbox_mode_choices,outputs=[maßnahmen_abfrage1_col, maßnahmen_abfrage2_col])
    ## Validation Checkboxes Choices Length
    checkbox_group_choices.change(validate_checkbox_choices_len, inputs=checkbox_group_choices,outputs=[error_box, submit_btn])
    ## Submit Button Event
    submit_btn.click(fn=make_prediction, inputs={in_landkreis_tb, school_close_intensity, workplace_close_intensity, cancel_events_intensity, gatherings_restrictions_intensity, transport_closing_intensity, \
                                                stay_home_restrictions_intensity, internal_movements_restrictions_intensity, international_movements_restrictions_intensity, information_campgaings_intensity, \
                                                testing_policy_intensity, contact_tracing_intensity, facial_coverings_intensity, vaccination_policy_intensity, elderly_people_protection_intensity, \
                                                in_inzidenz_bisher_tb_start, in_inzidenz_bisher_tb_ende}, outputs=[out_prognose_tb, out_inzidenz_hamburg_plot, out_inzidenz_ahrweiler_plot, out_inzidenz_integral_plot])
    # submit_btn.click(fn=make_prediction, inputs={in_landkreis_tb, school_close_cb, school_close_intensity, workplace_close_cb, workplace_close_intensity, in_inzidenz_bisher_tb_start, in_inzidenz_bisher_tb_ende}, outputs=out_prognose_tb)
    
demo.launch()

Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.
